# YouTube Machine Learning Model
#### Created by Randhir and Andrew

Model that will take a $90\times120$ thumbnail JPEG and title from YouTube to output a video performance metric.
The metric will be 
$$Score=\log{(View\ Count + 1)}$$
The idea is that the video that attracted more views is a good video. The value is log-scaled as the higher the view count, the less meaningful it becomes. This value will be normalized with the maximum value in the dataset.

##### Imports

In [ ]:
import string, re, requests, os, json, random
import urllib.request
import urllib.error
from pathlib import Path

from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

import numpy as np
import pandas as pd

from PIL import Image
import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from keras.backend import clear_session
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, TextVectorization, Embedding, Dropout, Concatenate, Input
from keras import Model
from sklearn.model_selection import KFold

from dotenv import load_dotenv

from datagen import ThumbnailDataGenerator

# Load .env file with your api key
load_dotenv()

True

#### Constants
This cell contains the constants used by this model

In [ ]:
# File Structure
dirpath = "thumbnail"
modeldir = "models"
datafile = "data-filtered.csv"

# Data Aquisition
filepath = "data.csv"
count = 50
max_iterations = 100 # 50 * 100 = 5000 videos
topic_id = "/m/03hf_rm" # Strategy Games
lang = "en"
API_KEY = os.getenv("APIKEY")

# Constants
MULT_CSV = False
filepath = "data.csv"
filepath_2 = "data_2.csv"
filepath_final = "data-filtered.csv"
lang = "en"

# Labeling
MAX_VIEWS = 15.3e9 # Baby Shark Video (Most Viewed Video)
THRESHOLD = 0.4

# Regex Patterns
emoji_re = "[\U000000A9-\U0010ffff]"
punc_re = f"[{re.escape(string.punctuation)}]"

# Download Stopwords & pattern
nltk.download('stopwords')
stopwords_list = stopwords.words("english")
sw_re = f'(?:{"|".join([f"{re.escape(sw)}" for sw in stopwords_list])})'

# Text Model Settings
text_input_dim = 20000
embedding_dim = 128
sequence_length = 500

# KFold Settings
n_folds = 5
epochs = 10
batch_size = 30

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Data Aquisition
The YouTube API is used to get video data. This includes a video's thumbnail and metrics.

In [ ]:
# Check if data file already exist
if os.path.isfile(filepath):
    df = pd.read_csv(filepath, index_col="yt-id")
else:
    df = pd.DataFrame([], columns=["yt-id", "title", "created", "channel-id", "thumbnail", "thumbnail-w", "thumbnail-h", "view-count", "like-count", "comment-count", "query"])
    df = df.set_index("yt-id")
    
# Grab missing data IDs for query
yt_ids = list(df[df["view-count"].isna()].index)

# Loop
yt_reads = 0
for i in range(max_iterations):
    try:
        # Check if any stats calls are needed
        if len(yt_ids) > 0:
            # Message 
            print("Pulling statistics for missing data values")

            # Split up batch by 50 if needed
            for index_split in range(50, len(yt_ids) + 1, 50):
                # Generate & call statistic query (1 unit)
                urlData_stats = f"https://www.googleapis.com/youtube/v3/videos?key={API_KEY}&part=statistics&id={','.join(yt_ids[index_split - 50:index_split])}"
                webURL_stats = urllib.request.urlopen(urlData_stats)
                raw_stats_data = webURL_stats.read()
                results_stats = json.loads(raw_stats_data.decode(webURL_stats.info().get_content_charset('utf-8')))

                # Process Stats Response
                for stats_data in results_stats["items"]:
                    try:
                        # Parse data
                        new_row = pd.DataFrame([{
                            "yt-id": stats_data['id'],
                            "view-count": stats_data['statistics']['viewCount'],
                            "like-count": stats_data['statistics']['likeCount'] if 'likeCount' in stats_data['statistics'] else "",
                            "comment-count": stats_data['statistics']['commentCount'] if 'commentCount' in stats_data['statistics'] else "",
                        },])
                        new_row = new_row.set_index("yt-id")

                        # Update main dataset
                        df.update(new_row)
                    except KeyError:
                        # Weird Entry
                        continue

            # Reset after used
            yt_ids = [] 

            # Message 
            print("Finished pulling statistics for current batch")

        # Message
        print(f"Pulling {count} random videos")

        # Generates random query for YT
        r_q = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(3))

        # Calls the API for search results (100 units)
        urlData_query = f"https://www.googleapis.com/youtube/v3/search?key={API_KEY}&maxResults={count}&part=snippet&type=video&relevanceLanguage={lang}&topicId={topic_id}&q={r_q}"
        webURL_query = urllib.request.urlopen(urlData_query)
        raw_vid_data = webURL_query.read()
        results_vids = json.loads(raw_vid_data.decode(webURL_query.info().get_content_charset('utf-8')))

        # Process Video Response
        for video_data in results_vids['items']:
            # Ignore Live and Upcoming Content (no ratings yet)
            if video_data['snippet']['liveBroadcastContent'] != "none":
                continue

            # Parse data
            try:
                new_row = pd.DataFrame([{
                    "yt-id": video_data['id']['videoId'],
                    "title": video_data['snippet']['title'],
                    "created": video_data['snippet']['publishedAt'],
                    "channel-id": video_data['snippet']['channelId'],
                    "thumbnail": video_data['snippet']['thumbnails']["default"]["url"],
                    "thumbnail-w": video_data['snippet']['thumbnails']["default"]["width"],
                    "thumbnail-h": video_data['snippet']['thumbnails']["default"]["height"],
                    "query": r_q,
                },])
                new_row = new_row.set_index("yt-id")

                try:
                    # Append
                    df = pd.concat([df, new_row], verify_integrity=True)

                    # Store your ids
                    yt_reads += 1

                    # Prepare id for stats query
                    yt_ids.append(video_data['id']['videoId'])
                except ValueError:
                    # Duplicate video detected
                    continue
            except KeyError:
                # Weird Entry
                continue

        # Update User
        print(f"API call #{i + 1} successfully")

        # Dumb Data to prevent loss every 5 runs
        if i % 5 == 0:
            df.to_csv(filepath)

    # ON API failure, quit and save
    except urllib.error.HTTPError:
        print("Latest API call failed. You are likely out of units. Try again tomorrow.")
        break
    
# Write to csv
df.to_csv(filepath)

# Termination
print(f"Was able to pull {yt_reads} rows")
del df

After aquiring the data, the thumbnail images need to be pulled as well.

#### Data Filtering
Some of the pulled data need to be filtered before usage. This includes potential duplicates and non english entries. 

In [ ]:
# Read Data
df = pd.read_csv(filepath, index_col="yt-id")

# Merge multiple if needed
if MULT_CSV:
	df_2 = pd.read_csv(filepath_2, index_col="yt-id")
	df = pd.concat([df, df_2])

print(f"{df.size} rows in data file")

# Remove duplicates
df = df[~df.index.duplicated(keep='first')]
print(f"{df.size} rows remaining after duplication filter")

# Remove non language
def lang_filter(row) -> bool:
	try:
		print(row["title"])
		return detect(row["title"]) == lang
	except LangDetectException:
		return False
	
df = df[df.apply(lang_filter, axis=1)]
print(f"{df.size} rows remaining after translation filter")

# Save Filtered Data
df.to_csv(filepath_final)
del df

#### Thumbnail Requesting
Once the dataset has been filtered, the thumbnails can now be pulled. Images that do not fit the $90\times120$ size will be rejected.

In [ ]:
# Grab data
df = pd.read_csv(filepath, index_col="yt-id")

# Make directory for image if not already
if not os.path.isdir(dirpath):
	os.mkdir(dirpath)

# Iterate thru dataframe and download
def grab_thumbnail(x : pd.Series):
	# Check if file exist
	filename = f'{dirpath}/{x.name}.jpg'
	if os.path.isfile(filename):
		print(f"Thumbnail already retrieved for {x.name}")
		return

	# Call file
	with open(filename, 'wb') as handle:
		print(f"Retrieving thumbnail for {x.name}")
		response = requests.get(x["thumbnail"], stream=True)

		# Fail request
		if not response.ok:
			print(f"Could not retrieve thumbnail for {x.name}")

		# Success save
		for block in response.iter_content(1024):
			if not block:
				break

			handle.write(block)

# Apply to all
df.apply(grab_thumbnail, axis=1)
del df

In [8]:
# Filter Images
files = [f for f in os.listdir(dirpath) if os.path.isfile(f"{dirpath}/{f}") and f.endswith(".jpg")]
image_ids = []
i = 0
for f in files:
	im = None
	try:
		im = Image.open(f"{dirpath}/{f}")

		if im.size != (120, 90):
			im.close()
			Path.unlink(f"{dirpath}/{f}")
			print(f"{f} deleted")
			continue

		im.close()
		im = None

		# Save valid indexes for filtering
		image_ids.append(f[:-4])
	except:
		# Close bad files
		if im is not None:
			im.close()
			im = None

		# Delete Bad Files
		Path.unlink(f"{dirpath}/{f}")
		print(f"{f} deleted")

	i += 1

02v-CVttnS0.jpg deleted
0EZUP5Vtemw.jpg deleted
4KlB4i4dEWU.jpg deleted
6JhUQpe-J6U.jpg deleted
bAHQy0QFUMI.jpg deleted
DcejDtVA4MU.jpg deleted
E0Hchyxwr4c.jpg deleted
ffLdLgSbpEc.jpg deleted
hstJLLvhYSM.jpg deleted
htE2M7shdfI.jpg deleted
JTwsU2dDpEg.jpg deleted
Lpnw6hMIu24.jpg deleted
Q9D-aQzRuU4.jpg deleted
RPoQZ_926hQ.jpg deleted
Sl2ueV8kRRU.jpg deleted
StkNJFSGksg.jpg deleted
tnAYVF1-q74.jpg deleted
VDg_U-n3t-I.jpg deleted
X82cgnMGeD8.jpg deleted
XO6KolPTH8U.jpg deleted
XY6Iw4kTOEI.jpg deleted
yQKNzY4HGGg.jpg deleted
ZBbw3WfcxN8.jpg deleted


#### Data Processing
This process involves text processing and image processing. This will involve text standardization and vectorization. For the image, it needs to be processed and normalized.

In [3]:
raw_data = pd.read_csv(datafile, index_col="yt-id")

# Filter raw data for thumbnail only entries
thumbnail_ids = np.array([f[:-4] for f in os.listdir(dirpath) if os.path.isfile(f"{dirpath}/{f}") and f.endswith(".jpg")], dtype=str)
raw_data = raw_data.loc[thumbnail_ids]

raw_data.describe()

,thumbnail-w,thumbnail-h,view-count,like-count,comment-count
count,35734.0,35734.0,3.527600e+04,3.428900e+04,35006.000000
mean,120.0,90.0,9.287259e+04,2.304083e+03,115.992858
std,0.0,0.0,1.687911e+06,3.117386e+04,1640.940831
min,120.0,90.0,0.000000e+00,0.000000e+00,0.000000
25%,120.0,90.0,3.000000e+01,1.000000e+00,0.000000
50%,120.0,90.0,2.815000e+02,8.000000e+00,1.000000
75%,120.0,90.0,3.153250e+03,7.500000e+01,12.000000
max,120.0,90.0,2.232996e+08,2.686147e+06,146332.000000


In [ ]:
# Text Processing
def text_standardization(raw_strs):
	lower = tf.strings.lower(raw_strs)
	emojiless = tf.strings.regex_replace(lower, emoji_re, "")
	stopwrdless = tf.strings.regex_replace(emojiless, sw_re, "")
	punctuationless = tf.strings.regex_replace(stopwrdless, punc_re, "")
	return punctuationless

# Vectorization Layer
vectorize_layer = TextVectorization(
    standardize=text_standardization,
    max_tokens=text_input_dim,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [19]:
# Label Processing
scores = raw_data["view-count"] # Grab View Count
scores = scores.fillna(0.0) # Replace NaN with 0
scores = scores.map(lambda x : np.log10(x + 1)) # Log everything to make it less extreme
scores = scores.div(np.log10(MAX_VIEWS + 1)) # Normalized (+1 to prevent one)

# Boolean Label
b_scores = scores.map(lambda x : int(x >= THRESHOLD))

scores.describe()

count    35734.000000
mean         0.251066
std          0.141486
min          0.000000
25%          0.142091
50%          0.237930
75%          0.341665
max          0.819749
Name: view-count, dtype: float64

##### Model

Commonly, the Sequential API is used to train a model. However, due to the need for more than one input, the Functional API must be employed.

In [15]:
# Image Portion
img_input = Input((90, 120, 3))
x = Conv2D(32, 5, activation='relu', padding='same')(img_input)
x = MaxPooling2D()(x)
x = Conv2D(64, 3, activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, 3, activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
img_output = Dense(1, activation='sigmoid')(x)

img_model = Model(inputs=img_input, outputs=img_output, name="img_model")

img_model.summary()

Model: "img_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 90, 120, 3)]      0         
                                                                 
 conv2d_9 (Conv2D)           (None, 90, 120, 32)       2432      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 45, 60, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 45, 60, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 22, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 22, 30, 128)       73

In [ ]:
# Text Portion
text_input = Input((1,), dtype=tf.string)
y = Embedding(text_input_dim, embedding_dim, input_length=)(text_input)
y = Dropout(0.5)(y)
y = Flatten()(y)
y = Dense(100, activation='relu', kernel_initializer='he_uniform')(y)
text_output = Dense(1, activation='sigmoid')(y)

text_model = Model(inputs=text_input, outputs=text_output, name="text_model")

text_model.summary()

Model: "text_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 embedding (Embedding)       (None, 1, 128)            2560000   
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 flatten_2 (Flatten)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               12900     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 2573001 (9.82 MB)
Trainable params: 257300

In [23]:
# United Model
z = Concatenate()([x, y])
z = Dense(10, activation='relu', kernel_initializer='he_uniform')(z)
z = Dense(1, activation='sigmoid')(z)

united_model = Model(inputs=[img_input, text_input], outputs=z, name="unitied_model")

united_model.summary()

Model: "unitied_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 90, 120, 3)]         0         []                            
                                                                                                  
 conv2d_3 (Conv2D)           (None, 88, 118, 16)          448       ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_3 (MaxPoolin  (None, 44, 59, 16)           0         ['conv2d_3[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_4 (Conv2D)           (None, 42, 57, 32)           4640      ['max_pooling2d_3[

#### Training
Using k-fold cross validation, we can judge the accuarcy of this model

In [ ]:
# Image Model
kf = KFold(n_folds)

validation_accuracy = []
validation_loss = []

fold_var = 1
for train, val in kf.split(thumbnail_ids, b_scores):
	# Fold Indicator
	print(f"Starting k-Fold #{fold_var}")

	# Make image model for testing
	img_model = Model(inputs=img_input, outputs=img_output, name="img_model")
	img_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

	# Callback Saving
	checkpoint = ModelCheckpoint(f"{modeldir}/model_{fold_var}.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

	# Generator
	tbdg_train = ThumbnailDataGenerator(dirpath, thumbnail_ids[train], b_scores.iloc[train], batch_size=batch_size)
	tbdg_validate = ThumbnailDataGenerator(dirpath, thumbnail_ids[val], b_scores.iloc[val], batch_size=batch_size)

	# Fit
	history = img_model.fit(x=tbdg_train, validation_data=tbdg_validate, callbacks=[checkpoint], epochs=epochs)

	# Grab Results
	img_model.load_weights(f"{modeldir}/model_{fold_var}.h5")
	
	results = img_model.evaluate(x=tbdg_validate)
	results = dict(zip(img_model.metrics_names, results))
	
	validation_accuracy.append(results['accuracy'])
	validation_loss.append(results['loss'])
	
	# Clear
	clear_session()

	# Increment
	fold_var += 1

Epoch 1/10
952/952 [==============================] - ETA: 0s - loss: 0.4072 - accuracy: 0.8365 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 1: val_accuracy improved from -inf to 0.86246, saving model to models\model_1.h5
952/952 [==============================] - 168s 175ms/step - loss: 0.4072 - accuracy: 0.8365 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.3592 - val_accuracy: 0.8625 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10


C:\Users\andre\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


952/952 [==============================] - ETA: 0s - loss: 0.3952 - accuracy: 0.8366 - precision: 0.5128 - recall: 0.0086
Epoch 2: val_accuracy did not improve from 0.86246
952/952 [==============================] - 166s 175ms/step - loss: 0.3952 - accuracy: 0.8366 - precision: 0.5128 - recall: 0.0086 - val_loss: 0.3559 - val_accuracy: 0.8625 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
952/952 [==============================] - ETA: 0s - loss: 0.3900 - accuracy: 0.8368 - precision: 0.5287 - recall: 0.0197
Epoch 3: val_accuracy improved from 0.86246 to 0.86261, saving model to models\model_1.h5
952/952 [==============================] - 163s 171ms/step - loss: 0.3900 - accuracy: 0.8368 - precision: 0.5287 - recall: 0.0197 - val_loss: 0.3557 - val_accuracy: 0.8626 - val_precision: 1.0000 - val_recall: 0.0010
Epoch 4/10
952/952 [==============================] - ETA: 0s - loss: 0.3841 - accuracy: 0.8362 - precision: 0.4897 - recall: 0.0358
Epoch 4: val_accuracy improve